In [4]:
import json
import re
import time

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
from tqdm import tqdm
import requests
import xmltodict

%reload_ext autoreload
%autoreload 2
sys.path.append('../scripts')
import pbmd_tools as pbmd

ModuleNotFoundError: No module named 'pbmd_tools'

## Last commit analysis

In [7]:
owner = 'guilledufort'
repo = 'RENANO'

files_changed = pbmd.get_last_commit_files(owner, repo, GITHUB_TOKEN)
if files_changed:
    print('Files changed in the last commit:\n')
    for file in files_changed:
        print(file)

NameError: name 'pbmd' is not defined